# pandas字符串处理

## pandas的字符串处理:
1. 使用方法: 先获取Series的str属性,然后在属性上调用函数
2. 只能在字符串列上使用,不能在其他数据类型上使用
3. DataFrame上没有str属性和方法
4. Series.str并不是原始python原始字符串, 而是自己的一套方法, 大部分和python的str类似

## Series.str字符串方法列表参考文档:
[文档](https://pandas.pydataorg/pandas-docs/stable/reference/series.html#string-handling 'pandas字符串官方文档')

## 代码示例内容:
1. 获取Series的str属性, 然后使用各种字符串处理函数
2. 使用str的startswith、contains等 得到bool值的Series 作为查询条件
3. 需要多次str处理的链式操作
4. 使用正则表达式的处理

# 读取数据

In [1]:
import pandas as pd

In [2]:
fpath = './data/weather/weater_beijing.xlsx'
df = pd.read_excel(fpath)

# 将最高温、最低温的°干掉, 并进行缺失值的处理
# 处理缺失值第一种方式
df.loc[:, '最高温'] = df['最高温'].str.replace('°', '').str.replace('', '0')
df['最高温'].fillna('0', inplace=True)
df.loc[:, '最高温'] = df['最高温'].astype(int)
# 处理缺失值第二种方式
df.loc[:, '最低温'] = df['最低温'].str.replace('°', '').str.replace('', '0')
df.loc[:, '最低温'] = df['最低温'].fillna({'最低温': '0'})
df.loc[:, '最低温'] = df['最低温'].astype(int)

# 1. 获取Series的str属性, 使用各种字符串处理函数

In [3]:
df.dtypes

日期        object
最高温       object
最低温       object
天气        object
风力风向      object
空气质量指数    object
dtype: object

In [4]:
df['天气'].str

In [5]:
# 判断是不是数字
df['天气'].str.isnumeric()

0       False
1       False
2       False
3       False
4       False
        ...  
4001    False
4002    False
4003    False
4004    False
4005    False
Name: 天气, Length: 4006, dtype: bool

In [6]:
df['天气'].str.len()

0       4
1       2
2       4
3       1
4       1
       ..
4001    1
4002    4
4003    1
4004    1
4005    1
Name: 天气, Length: 4006, dtype: int64

In [7]:
df

,日期,最高温,最低温,天气,风力风向,空气质量指数
0,2011-01-01 周六,-2,-7,多云~阴,无持续风向微风,NaN
1,2011-01-02 周日,-2,-7,多云,无持续风向微风,NaN
2,2011-01-03 周一,-2,-6,多云~阴,西北风~北风3-4级~4-5级,NaN
3,2011-01-04 周二,-2,-9,晴,北风5-6级,NaN
4,2011-01-05 周三,-2,-10,晴,北风~无持续风向3-4级~微风,NaN
...,...,...,...,...,...,...
4001,2021-12-27 周一,6,-8,晴,西北风1级,56 良
4002,2021-12-28 周二,6,-5,多云~晴,西北风1级,64 良
4003,2021-12-29 周三,5,-5,晴,西北风3级,43 优
4004,2021-12-30 周四,6,-7,晴,西北风3级,38 优


In [8]:
# 将空气质量指数这一列处理, 做缺失值处理后,将数据进行拆分成两列 空气质量(优、良) 空气质量指数
df['空气质量指数'].fillna('-1 未统计', inplace=True)

# 第一种方式
df_copy = df.copy(True)
df_copy[['空气质量','空气质量指数']] = df_copy['空气质量指数'].apply(lambda x: pd.Series(x.split(" ")))

In [9]:
df_copy

,日期,最高温,最低温,天气,风力风向,空气质量指数,空气质量
0,2011-01-01 周六,-2,-7,多云~阴,无持续风向微风,未统计,-1
1,2011-01-02 周日,-2,-7,多云,无持续风向微风,未统计,-1
2,2011-01-03 周一,-2,-6,多云~阴,西北风~北风3-4级~4-5级,未统计,-1
3,2011-01-04 周二,-2,-9,晴,北风5-6级,未统计,-1
4,2011-01-05 周三,-2,-10,晴,北风~无持续风向3-4级~微风,未统计,-1
...,...,...,...,...,...,...,...
4001,2021-12-27 周一,6,-8,晴,西北风1级,良,56
4002,2021-12-28 周二,6,-5,多云~晴,西北风1级,良,64
4003,2021-12-29 周三,5,-5,晴,西北风3级,优,43
4004,2021-12-30 周四,6,-7,晴,西北风3级,优,38


In [10]:
# 第二种方式
def split_data(d):
   r = pd.Series(d.split(' '))
   return r


df_copy1 = df.copy(True)
df_copy1[['空气质量','空气质量指数']] = df_copy1['空气质量指数'].apply(split_data)

In [11]:
df_copy1

,日期,最高温,最低温,天气,风力风向,空气质量指数,空气质量
0,2011-01-01 周六,-2,-7,多云~阴,无持续风向微风,未统计,-1
1,2011-01-02 周日,-2,-7,多云,无持续风向微风,未统计,-1
2,2011-01-03 周一,-2,-6,多云~阴,西北风~北风3-4级~4-5级,未统计,-1
3,2011-01-04 周二,-2,-9,晴,北风5-6级,未统计,-1
4,2011-01-05 周三,-2,-10,晴,北风~无持续风向3-4级~微风,未统计,-1
...,...,...,...,...,...,...,...
4001,2021-12-27 周一,6,-8,晴,西北风1级,良,56
4002,2021-12-28 周二,6,-5,多云~晴,西北风1级,良,64
4003,2021-12-29 周三,5,-5,晴,西北风3级,优,43
4004,2021-12-30 周四,6,-7,晴,西北风3级,优,38


# 2. 使用str的startswith、contains等 得到bool值的Series 作为查询条件

In [12]:
condition = df['日期'].str.startswith('2018-03')

In [13]:
condition

0       False
1       False
2       False
3       False
4       False
        ...  
4001    False
4002    False
4003    False
4004    False
4005    False
Name: 日期, Length: 4006, dtype: bool

In [14]:
df[condition].head()

,日期,最高温,最低温,天气,风力风向,空气质量指数
2604,2018-03-01 周四,8,-3,多云,西南风1-2级,46 优
2605,2018-03-02 周五,9,-1,晴~多云,北风1-2级,95 良
2606,2018-03-03 周六,13,3,多云~阴,北风1-2级,214 重度
2607,2018-03-04 周日,7,-2,阴~多云,东南风1-2级,144 轻度
2608,2018-03-05 周一,8,-3,晴,南风1-2级,94 良


# 3. 需要多次str处理的链式操作
怎样提前201803这样的数字月份
1. 先将日期2018-03-31形式替换为20180331的形式
2. 提取月份字符串201803

In [15]:
df['日期'].str.replace('-', '')

0       20110101 周六
1       20110102 周日
2       20110103 周一
3       20110104 周二
4       20110105 周三
           ...     
4001    20211227 周一
4002    20211228 周二
4003    20211229 周三
4004    20211230 周四
4005    20211231 周五
Name: 日期, Length: 4006, dtype: object